<a href="https://colab.research.google.com/github/charliesire/FunQuant/blob/main/short_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install.packages("remotes")
remotes::install_github("CharlieSire/FunQuant")
library(FunQuant)

install.packages("randtoolbox")
library(randtoolbox)
install.packages("evd")
library(evd)

ERROR: ignored

We will investigate the function $f(x) = sin(x_{1})*sin(x_{2})$

We can introduce the design of experiments

In [ ]:
design = as.data.frame(halton(400,2))*2-1
outputs = apply(sin(design), 1, prod)

We can first perform quantization without considering the probabilistic distribution of the inputs. This is classical K-means quantization.

In [ ]:

```{r}

res_proto = proto_map_algo(nb_cells = 4, outputs = outputs, multistart = 6) 

res_proto
```
We can also consider the probabilistic distribution of the inputs. 

```{r}
f2 = function(x){
  res = 0
  ptrunc = pgev(-1, loc=-0.4,scale=0.1) + 1 - pgev(1, loc=-0.4,scale=0.1)
  if(x>=-1 & x < 1){res = dgev(x, loc=-0.4,scale=0.1)/(1-ptrunc)}
  return(res)
}
f1 = function(x){
  if(x > -1 & x < 1){return(1/2)}
  else{return(0)}
}
fX = function(x){f1(x[1])*f2(x[2])}
```

The design of experiments is a sobol sequence that can be considered as a uniform distribution, which is often the case. 

```{r}
g = function(x){
  if(sum(x > -1) == length(x) & sum(x < 1) == length(x)){return(1/4)}
  else{return(0)}
}
```

Then we can perform a probabilistic quantization

```{r}
density_ratio = compute_density_ratio(f = fX, g = g, inputs = design)
res_proto_2 = proto_map_algo(nb_cells = 4, outputs = outputs, multistart = 6,density_ratio = density_ratio) 


res_proto_2
```
